# Azure AI Agents met ondersteuning voor Model Context Protocol (MCP) - Python

Dit notebook laat zien hoe je Azure AI Agents kunt gebruiken met Model Context Protocol (MCP)-tools in Python. Het toont hoe je een intelligente agent kunt maken die externe MCP-servers (zoals Microsoft Learn) kan benutten voor verbeterde mogelijkheden met behulp van sleutelvrije authenticatie.


## Vereiste Python-pakketten installeren

Eerst moeten we de benodigde Python-pakketten installeren:
- **azure-ai-projects**: Kern-SDK voor Azure AI Projects
- **azure-ai-agents**: Azure AI Agents SDK voor het maken en beheren van agents
- **azure-identity**: Biedt sleutelvrije authenticatie met behulp van DefaultAzureCredential
- **mcp**: Implementatie van het Model Context Protocol voor Python


## Voordelen van Keyless Authenticatie

Deze notebook demonstreert **keyless authenticatie**, wat verschillende voordelen biedt:
- ✅ **Geen API-sleutels om te beheren** - Maakt gebruik van Azure identity-gebaseerde authenticatie
- ✅ **Verbeterde beveiliging** - Geen geheimen opgeslagen in code of configuratiebestanden
- ✅ **Automatische rotatie van referenties** - Azure beheert de levenscyclus van referenties
- ✅ **Toegangscontrole op basis van rollen** - Maakt gebruik van Azure RBAC voor gedetailleerde machtigingen
- ✅ **Ondersteuning voor meerdere omgevingen** - Werkt naadloos in zowel ontwikkel- als productieomgevingen

De `DefaultAzureCredential` selecteert automatisch de best beschikbare bron voor referenties:
1. **Managed Identity** (bij uitvoering in Azure)
2. **Azure CLI** referenties (tijdens lokale ontwikkeling)
3. **Visual Studio** referenties
4. **Omgevingsvariabelen** (indien geconfigureerd)
5. **Interactieve browser** authenticatie (als laatste redmiddel)


## Instellen van sleutelvrije authenticatie

**Vereisten voor sleutelvrije authenticatie:**

### Voor lokale ontwikkeling:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Voor Azure-omgevingen:
- Schakel **Systeemtoegewezen Managed Identity** in op je Azure-resource
- Wijs de juiste **RBAC-rollen** toe aan de managed identity:
  - `Cognitive Services OpenAI User` voor toegang tot Azure OpenAI
  - `AI Developer` voor toegang tot Azure AI-projecten

### Omgevingsvariabelen (Optioneel):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Geen API-sleutels of verbindingsstrings nodig!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Vereiste Bibliotheken Importeren

Importeer de benodigde Python-modules:
- **os, time**: Standaard Python-bibliotheken voor omgevingsvariabelen en vertragingen
- **AIProjectClient**: Hoofdclient voor Azure AI-projecten
- **DefaultAzureCredential**: Sleutelloze authenticatie voor Azure-diensten
- **MCP-gerelateerde klassen**: Voor het maken en beheren van MCP-tools en het afhandelen van goedkeuringen


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## MCP-serverinstellingen configureren

Stel de MCP-serverconfiguratie in met behulp van omgevingsvariabelen met standaardwaarden als fallback:
- **MCP_SERVER_URL**: De URL van de MCP-server (standaard de Microsoft Learn API)
- **MCP_SERVER_LABEL**: Een label om de MCP-server te identificeren (standaard "mslearn")

Deze aanpak maakt flexibele configuratie mogelijk in verschillende omgevingen.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Azure AI Project Client maken (Keyless Authenticatie)

Initialiseer de Azure AI Project client met **keyless authenticatie**:
- **endpoint**: De URL van het Azure AI Foundry project endpoint
- **credential**: Gebruikt `DefaultAzureCredential()` voor veilige, keyless authenticatie
- **Geen API-sleutels nodig**: Ontdekt en gebruikt automatisch de best beschikbare credential

**Authenticatieproces:**
1. Controleert op Managed Identity (in Azure-omgevingen)
2. Valt terug op Azure CLI-credentials (voor lokale ontwikkeling)
3. Gebruikt andere beschikbare credential bronnen indien nodig

Deze aanpak elimineert de noodzaak om API-sleutels of verbindingsstrings in je code te beheren.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Maak MCP Tool Definitie

Maak een MCP-tool die verbinding maakt met de Microsoft Learn MCP-server:
- **server_label**: Identificatie voor de MCP-server
- **server_url**: URL-eindpunt van de MCP-server
- **allowed_tools**: Optionele lijst om te beperken welke tools gebruikt kunnen worden (lege lijst staat alle tools toe)

Deze tool stelt de agent in staat om toegang te krijgen tot Microsoft Learn documentatie en bronnen.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Maak Agent en Voer Gesprek Uit (Keyless Workflow)

Deze uitgebreide sectie demonstreert de volledige **keyless agent workflow**:

1. **Maak AI Agent**: Stel een agent in met het GPT-4.1 nano model en MCP-tools
2. **Maak Thread**: Creëer een gespreksthread voor communicatie
3. **Verstuur Bericht**: Vraag de agent naar de verschillen tussen Azure OpenAI en OpenAI
4. **Behandel Tool Goedkeuringen**: Keur automatisch MCP-tool oproepen goed wanneer nodig
5. **Monitor Uitvoering**: Volg de voortgang van de agent en voer eventuele vereiste acties uit
6. **Toon Resultaten**: Laat het gesprek en de details van het gebruik van tools zien

**Keyless Kenmerken:**
- ✅ **Geen hardcoded geheimen** - Alle authenticatie wordt afgehandeld door Azure-identiteit
- ✅ **Standaard veilig** - Maakt gebruik van rolgebaseerde toegangscontrole
- ✅ **Eenvoudige implementatie** - Geen beheer van inloggegevens nodig
- ✅ **Audit-vriendelijk** - Alle toegang wordt gevolgd via Azure-identiteit

De agent zal MCP-tools gebruiken om toegang te krijgen tot Microsoft Learn-bronnen met volledige beveiliging en zonder API-sleutelbeheer.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we ons best doen voor nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
